In [1]:
# primary settings
state = 'AL'
district_type = 'CD'
level = 'block'
minority = 'Black'

In [2]:
filepath = 'C:\\districting-data-2020\\'
filename = state + '_' + level + '.json'

from read import read_graph_from_json
G = read_graph_from_json( filepath + filename )
print("number of nodes, edges:",G.number_of_nodes(),G.number_of_edges())

number of nodes, edges: 185976 461910


In [3]:
# read Plan D from file
filename = "PlanDDec26.csv"
geoid_to_node = { G.nodes[i]['GEOID20'] : i for i in G.nodes }

from number_of_districts import number_of_districts
k = number_of_districts[state, district_type]
districts = [ list() for j in range(k) ]

import pandas
csvFile = pandas.read_csv( filename, skipinitialspace = True )
unassigned = list()
for index, row in csvFile.iterrows():

    # what is the block's geoid?
    g = str( row['GEOID20'] )
    if len(g) < 15: # fix issue with leading zeros
        g = '0' + g

    # which node does this correspond to?
    i = geoid_to_node[g]

    # which district is it in?
    j = str( row['District'] ) 

    # is it unassigned??
    if j in { 'ZZ', 'ZZZ' }:
        unassigned.append(i)
        continue
    else:
        districts[int(j)-1].append(i)

assert len(unassigned)==0

In [4]:
# read more-detailed version of block-level graph
#   to get additional data fields (VAP and MVAP)
GB = read_graph_from_json( "al_blocks.json" )

for i in G.nodes:
    #assert G.nodes[i]['GEOID20'] == GB.nodes[i]['GEOID20']
    G.nodes[i]['TOTPOP'] = G.nodes[i]['P0010001'] 
    G.nodes[i]['VAP'] = GB.nodes[i]['VAP20']
    G.nodes[i]['MVAP'] = GB.nodes[i]['APBVAP20']

del GB # free up some memory
    
from metrics import report_metrics
report_metrics(G, districts, minority=minority, verbose=True)


District Black percentages:
0 15.36 
1 50.05 ***
2 23.96 
3 8.58 
4 16.02 
5 15.37 
6 51.73 ***
-> 2 majority-Black districts

Counties split (by fips code): ['01113', '01083', '01073', '01097', '01117']
County splits (by fips code): {'01113': 1, '01083': 1, '01073': 2, '01097': 1, '01117': 1}
-> 5 counties split a total of 6 times

District Polsby-Popper scores:
0 0.1277
1 0.1532
2 0.2664
3 0.3609
4 0.3743
5 0.1891
6 0.2681
-> average Polsby-Popper score of 0.2485


In [5]:
populations = [ sum( G.nodes[i]['TOTPOP'] for i in district ) for district in districts ]
G._L = min(populations)
G._U = max(populations)
G._k = k
G._state = state
G._level = level
print("Using k, L, U =",G._k, G._L, G._U)

Using k, L, U = 7 717754 717755


In [6]:
# Cleanup the seed PlanD, making it more compact, 
#   with 1-hop and 2-hop local search
from cleanup import mip_local_search
from export import export_to_csv

In [7]:
districts = mip_local_search(G, districts, h=1, minority=minority)
report_metrics(G, districts, minority, verbose=True)

labeling = { i : j for j in range(len(districts)) for i in districts[j] }
csv_filename = "PlanD1.csv"
export_to_csv(G, labeling, csv_filename=csv_filename)

Applying MIP-based local search to improve the MIP warm start (hop parameter: 1)...
iter 		 obj 		 time
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-16
1 	 4.36731262 	 600.55
2 	 4.27758732 	 600.58
3 	 4.25808928 	 600.50
4 	 4.25192777 	 601.46
5 	 4.25015523 	 601.33
6 	 4.24830797 	 601.61
7 	 4.24287040 	 600.54
8 	 4.23669934 	 601.26
9 	 4.23514396 	 600.72
10 	 4.23514396 	 600.30

District Black percentages:
0 15.41 
1 50.0 ***
2 23.95 
3 8.58 
4 16.01 
5 15.39 
6 51.71 ***
-> 2 majority-Black districts

Counties split (by fips code): ['01113', '01083', '01073', '01097', '01117']
County splits (by fips code): {'01113': 1, '01083': 1, '01073': 2, '01097': 1, '01117': 1}
-> 5 counties split a total of 6 times

District Polsby-Popper scores:
0 0.1366
1 0.1648
2 0.2774
3 0.4303
4 0.4345
5 0.2114
6 0.3033
-> average Polsby-Popper score of 0.2798
Exporting to csv...


In [8]:
districts = mip_local_search(G, districts, h=2, minority=minority)
report_metrics(G, districts, minority, verbose=True)

labeling = { i : j for j in range(len(districts)) for i in districts[j] }
csv_filename = "PlanD2.csv"
export_to_csv(G, labeling, csv_filename=csv_filename)

Applying MIP-based local search to improve the MIP warm start (hop parameter: 2)...
iter 		 obj 		 time
1 	 4.10817541 	 601.56
2 	 4.04836814 	 602.01
3 	 4.02790996 	 609.05
4 	 4.01344333 	 601.68
5 	 4.00762234 	 601.90
6 	 4.00075543 	 601.17
7 	 3.99854768 	 601.34
8 	 3.99792512 	 601.25
9 	 3.99788503 	 600.97
10 	 3.99788503 	 602.34

District Black percentages:
0 15.39 
1 50.01 ***
2 23.95 
3 8.43 
4 16.01 
5 15.58 
6 51.64 ***
-> 2 majority-Black districts

Counties split (by fips code): ['01113', '01083', '01073', '01097', '01117']
County splits (by fips code): {'01113': 1, '01083': 1, '01073': 2, '01097': 1, '01117': 1}
-> 5 counties split a total of 6 times

District Polsby-Popper scores:
0 0.1418
1 0.1705
2 0.2844
3 0.4986
4 0.4735
5 0.2304
6 0.3233
-> average Polsby-Popper score of 0.3032
Exporting to csv...


In [10]:
districts = mip_local_search(G, districts, h=3, minority=minority, time_limit=3600)
report_metrics(G, districts, minority, verbose=True)

labeling = { i : j for j in range(len(districts)) for i in districts[j] }
csv_filename = "PlanD3.csv"
export_to_csv(G, labeling, csv_filename=csv_filename)

Applying MIP-based local search to improve the MIP warm start (hop parameter: 3)...
iter 		 obj 		 time
1 	 3.92869988 	 3602.07
2 	 3.90388836 	 3605.59
3 	 3.83888808 	 3609.51
4 	 3.83780845 	 3620.04
5 	 3.83780845 	 3618.30

District Black percentages:
0 15.42 
1 50.0 ***
2 23.93 
3 8.33 
4 16.01 
5 15.71 
6 51.61 ***
-> 2 majority-Black districts

Counties split (by fips code): ['01113', '01083', '01073', '01097', '01117']
County splits (by fips code): {'01113': 1, '01083': 1, '01073': 2, '01097': 1, '01117': 1}
-> 5 counties split a total of 6 times

District Polsby-Popper scores:
0 0.1467
1 0.1727
2 0.2963
3 0.5382
4 0.4735
5 0.2586
6 0.3285
-> average Polsby-Popper score of 0.3164
Exporting to csv...


In [11]:
districts = mip_local_search(G, districts, h=4, minority=minority, time_limit=3600)
report_metrics(G, districts, minority, verbose=True)

labeling = { i : j for j in range(len(districts)) for i in districts[j] }
csv_filename = "PlanD4.csv"
export_to_csv(G, labeling, csv_filename=csv_filename)

Applying MIP-based local search to improve the MIP warm start (hop parameter: 4)...
iter 		 obj 		 time
1 	 3.80092146 	 3604.60
2 	 3.76924650 	 3601.63
3 	 3.75071796 	 3605.53
4 	 3.73930249 	 3602.97
5 	 3.72666108 	 3605.89
6 	 3.72638241 	 3602.69
7 	 3.72638240 	 3602.92

District Black percentages:
0 15.44 
1 50.0 ***
2 23.94 
3 8.19 
4 16.15 
5 15.7 
6 51.61 ***
-> 2 majority-Black districts

Counties split (by fips code): ['01113', '01083', '01073', '01097', '01117']
County splits (by fips code): {'01113': 1, '01083': 1, '01073': 2, '01097': 1, '01117': 1}
-> 5 counties split a total of 6 times

District Polsby-Popper scores:
0 0.1582
1 0.1779
2 0.2979
3 0.5449
4 0.4862
5 0.2599
6 0.3285
-> average Polsby-Popper score of 0.3219
Exporting to csv...
